In [1]:
#! pip install pandas

In [1]:
import pandas as pd
import time
import os
from decimal import Decimal


In [2]:
# Carga de datos
price = pd.read_csv('Data/PriceCurve_SE4_2021.csv', sep = ';')
co2_pro = pd.read_csv('Data/production_emissions.csv')
co2_con = pd.read_csv('Data/consumption_emissions.csv')
pv = pd.read_csv('Data/oskarshamnpvprod.csv')
data = pd.read_csv('Data/LoadCurve_new.csv', sep=';')
data['Load'] = data['Load'].apply(lambda x: x.replace(',', '.')).astype(float)
#test test test

In [3]:


# Preparación de datos
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

# MKWh a MWh
data['Load'] = data['Load'] * 1000
# Los datos solares ya están en kW, pero necesitan ser multiplicados por el factor de escala
#para la nueva data se multiplica por 1000
data['solar_PV']= (data['solar_PV'] * 10000)
data['Price'] = data['Price'] / 1000
data['CO_2_eq'] = data['CO_2_eq'] / 1000
data['Hour'] = data['Hour'].astype('int')

# Parámetros principales
params = {
    'pv_price': 80,                # https://data.nrel.gov/submissions/53 in EUR/kW
    'bess_price': 200,             # https://doi.org/10.1016/j.solener.2018.08.061 in EUR/kWh, adjusted for price decreases
    'bess_bos': 250,               # https://www.energy-storage.news/li-ion-bess-costs-could-fall-47-by-2030-nrel-says-in-long-term-forecast-update/ in EUR/kW
                                   # Also see: https://www.nrel.gov/grid/assets/pdfs/second_grid_sim_zagoras.pdf
    'pv_opex': 3,                  # EUR/kWh ->reference in excel
    'bess_opex': 6,                # EUR/kWh ->reference in excel
    'pv_co2': 33,                  # kgCO2eq/kW_powerDC ->reference in excel
    'bess_co2': 100,               # kgCO2eq/kWh_capacity ->reference in excel
    'pv_opex_co2': 0,              # kgCO2eq/kW_powerDC ->assumption
    'bess_opex_co2': 0,            # kgCO2eq/kW_powerDC ->assumption
    'discount_rate': 0.0485,       # assumption
    'degradation_rate': 0.025,     # assumption (based on reaching 80% SoH in 8 years)
    'lifetime_project': 32,        # for the project lifetime
    'lifetime_bess': 8             # for the BESS lifetime
}

def calc_lcoe(annual_output, capital_cost, annual_operating_cost, eol_burden, discount_rate, degradation_rate, lifetime):
    discount_factor_annualised = ((1 - (1 / ((1 + discount_rate) ** lifetime))) * (1 + discount_rate)) / ((1 + discount_rate) - 1)
    degradation_factor_annualised = ((1 - (1 / ((1 + degradation_rate) ** lifetime))) * (1 + degradation_rate)) / ((1 + degradation_rate) - 1)
    result = (capital_cost + annual_operating_cost * discount_factor_annualised + eol_burden) / (annual_output * degradation_factor_annualised)
    return result

def get_KPIs(profile, solar_factor, PessCH, EnESS):
    price_curve = pd.read_csv('Data/PriceCurve_SE4_2021.csv', delimiter=';', usecols=['Grid_Price']) / 1000         #EUR/MWh
    co2_curve = pd.read_csv('Data/production_emissions.csv', delimiter=',', usecols=['carbon_intensity_production_avg']) / 1000     #gCO2/kWh OR kgCO2/MWh

    t_bess = EnESS / PessCH # Duración de la carga de la bateria


    hours_x_year = 8736 # Algunos archivos no llegan a 8760 horas anuales, por lo que da error y lo bajé a 8736 horas (1dia menos). otra opcion seria completarlo con 0s

    savings = sum((price_curve.Grid_Price[i] * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i])) for i in range(hours_x_year))
    energy_procurement_costs = sum((price_curve.Grid_Price[i] * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i])) for i in range(hours_x_year))
    energy_procurement_costs_load = sum((price_curve.Grid_Price[i] * profile.P_Grid_to_Load[i]) for i in range(hours_x_year))
    energy_procurement_costs_bess = sum((price_curve.Grid_Price[i] * profile.P_Grid_to_BESS[i]) for i in range(hours_x_year))
    profit = sum((price_curve.Grid_Price[i] * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i])) for i in range(hours_x_year))
    profit_pv = sum((price_curve.Grid_Price[i] * profile.P_PV_to_Grid[i]) for i in range(hours_x_year))
    profit_bess = sum((price_curve.Grid_Price[i] * profile.P_BESS_to_Grid[i]) for i in range(hours_x_year))
    co2avoided = sum((co2_curve.carbon_intensity_production_avg[i] * (profile.P_PV_to_Load[i] + profile.P_BESS_to_Load[i])) for i in range(hours_x_year))
    co2burden = sum((co2_curve.carbon_intensity_production_avg[i] * (profile.P_Grid_to_Load[i] + profile.P_Grid_to_BESS[i])) for i in range(hours_x_year))
    co2burden_load = sum((co2_curve.carbon_intensity_production_avg[i] * profile.P_Grid_to_Load[i]) for i in range(hours_x_year))
    co2burden_bess = sum((co2_curve.carbon_intensity_production_avg[i] * profile.P_Grid_to_BESS[i]) for i in range(hours_x_year))
    co2abatement = sum((co2_curve.carbon_intensity_production_avg[i] * (profile.P_BESS_to_Grid[i] + profile.P_PV_to_Grid[i])) for i in range(hours_x_year))
    co2abatement_pv = sum((co2_curve.carbon_intensity_production_avg[i] * profile.P_PV_to_Grid[i]) for i in range(hours_x_year))
    co2abatement_bess = sum((co2_curve.carbon_intensity_production_avg[i] * profile.P_BESS_to_Grid[i]) for i in range(hours_x_year))

    pv2load = profile['P_PV_to_Load'].sum()
    bess2load = profile['P_BESS_to_Load'].sum()
    pv2grid = profile['P_PV_to_Grid'].sum()
    bess2grid = profile['P_BESS_to_Grid'].sum()
    grid2load = profile['P_Grid_to_Load'].sum()
    pv2bess = profile['P_PV_to_BESS'].sum()
    pv2curtail = profile['P_PV_curtailment'].sum()    
    grid2bess = profile['P_Grid_to_BESS'].sum()

    annual_energy = pv2load + bess2load + pv2grid + bess2grid
    annual_energy_enduser = pv2load + bess2load + grid2load

    cycles = int((bess2load + bess2grid) / (EnESS * 1000)) if EnESS != 0 else 0

    capex = solar_factor * params['pv_price'] * 1000 + EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    capex_pv = solar_factor * params['pv_price'] * 1000
    capex_bess = EnESS * 1000 * params['bess_price'] + PessCH * 1000 * params['bess_bos']
    opex = solar_factor * params['pv_opex'] * 1000 + EnESS * 1000 * params['bess_opex']
    opex_pv = solar_factor * params['pv_opex'] * 1000
    opex_bess = EnESS * 1000 * params['bess_opex']
    opex_investor = opex + energy_procurement_costs_bess
    opex_enduser = opex + energy_procurement_costs_load + (energy_procurement_costs_bess * (bess2load / (bess2load + bess2grid)) if EnESS != 0 else 0)
    eol_capex = capex * 0.3
    capex_co2 = solar_factor * params['pv_co2'] * 1000 + EnESS * 1000 * params['bess_co2']
    capex_co2_pv = solar_factor * params['pv_co2'] * 1000
    capex_co2_bess = EnESS * 1000 * params['bess_co2']
    opex_co2 = solar_factor * params['pv_opex_co2'] * 1000 + EnESS * 1000 * params['bess_opex_co2']
    opex_co2_pv = solar_factor * params['pv_opex_co2'] * 1000
    opex_co2_bess = EnESS * 1000 * params['bess_opex_co2']
    opex_co2_investor = opex_co2 + co2burden_bess
    opex_co2_enduser = opex_co2 + co2burden_load + (co2burden_bess * (bess2load / (bess2load + bess2grid)) if EnESS != 0 else 0)
    eol_co2 = capex_co2 * 0.3

    lcoe_revenue = calc_lcoe(annual_energy, capex, opex + energy_procurement_costs_bess - profit, eol_capex, params['discount_rate'], params['degradation_rate'], params['lifetime_project']) if annual_energy != 0 else 0
    lcoe_investor = calc_lcoe(annual_energy, capex, opex_investor, eol_capex, params['discount_rate'], params['degradation_rate'], params['lifetime_project']) if annual_energy != 0 else 0
    lco2_abatement = calc_lcoe(annual_energy, capex_co2, opex_co2 + co2burden_bess - co2abatement, eol_co2, params['discount_rate'], params['degradation_rate'], params['lifetime_project']) if annual_energy != 0 else 0
    lco2_investor = calc_lcoe(annual_energy, capex_co2, opex_co2_investor, eol_co2, params['discount_rate'], params['degradation_rate'], params['lifetime_project']) if annual_energy != 0 else 0
    
    lcoe_enduser = calc_lcoe(annual_energy_enduser, capex, opex_enduser, eol_capex, params['discount_rate'], params['degradation_rate'], params['lifetime_project'])
    lco2_enduser = calc_lcoe(annual_energy_enduser, capex_co2, opex_co2_enduser, eol_co2, params['discount_rate'], params['degradation_rate'], params['lifetime_project'])

    self_sufficiency = ((pv2load + bess2load) / data.Load.sum()) * 100
    ress = ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100 if EnESS != 0 else (pv2load / data.Load.sum()) * 100
    self_consumption = ((pv2load + pv2bess) / (data.solar_PV.sum() * (solar_factor * 1000))) * 100 if solar_factor != 0 else 0

    parameters = (
        solar_factor, PessCH, EnESS, t_bess, pv2load, pv2bess, pv2curtail, pv2grid, bess2load, bess2grid, grid2load, grid2bess,
        annual_energy, capex, capex_pv, capex_bess, opex, opex_pv, opex_bess, capex_co2, capex_co2_pv, capex_co2_bess, opex_co2,
        opex_co2_pv, opex_co2_bess, lcoe_investor, lcoe_enduser, lcoe_revenue, lco2_investor, lco2_enduser, lco2_abatement,
        savings, energy_procurement_costs, energy_procurement_costs_load, energy_procurement_costs_bess, profit, profit_pv,
        profit_bess, co2avoided, co2burden, co2burden_load, co2burden_bess, co2abatement, co2abatement_pv, co2abatement_bess,
        cycles, self_sufficiency, ress, self_consumption
    )
    return parameters

results_econ = pd.DataFrame(columns=[
    'P_PV', 'P_BESS', 'E_BESS', 'T_BESS', 'pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load',
    'grid2bess', 'annual_energy', 'capex', 'capex_pv', 'capex_bess', 'opex', 'opex_pv', 'opex_bess', 'capex_co2', 'capex_co2_pv',
    'capex_co2_bess', 'opex_co2', 'opex_co2_pv', 'opex_co2_bess', 'lcoe_investor', 'lcoe_enduser', 'lcoe_revenue', 'lco2_investor',
    'lco2_enduser', 'lco2_abatement', 'savings', 'energy_procurement_costs', 'energy_procurement_costs_load', 'energy_procurement_costs_bess',
    'profit', 'profit_pv', 'profit_bess', 'co2avoided', 'co2burden', 'co2burden_load', 'co2burden_bess', 'co2abatement', 'co2abatement_pv',
    'co2abatement_bess', 'cycles', 'self_sufficiency', 'ress', 'self_consumption'
])

results_env = pd.DataFrame(columns=[
    'P_PV', 'P_BESS', 'E_BESS', 'T_BESS', 'pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load',
    'grid2bess', 'annual_energy', 'capex', 'capex_pv', 'capex_bess', 'opex', 'opex_pv', 'opex_bess', 'capex_co2', 'capex_co2_pv',
    'capex_co2_bess', 'opex_co2', 'opex_co2_pv', 'opex_co2_bess', 'lcoe_investor', 'lcoe_enduser', 'lcoe_revenue', 'lco2_investor',
    'lco2_enduser', 'lco2_abatement', 'savings', 'energy_procurement_costs', 'energy_procurement_costs_load', 'energy_procurement_costs_bess',
    'profit', 'profit_pv', 'profit_bess', 'co2avoided', 'co2burden', 'co2burden_load', 'co2burden_bess', 'co2abatement', 'co2abatement_pv',
    'co2abatement_bess', 'cycles', 'self_sufficiency', 'ress', 'self_consumption'
])




In [4]:
optimise_for = ['price', 'co2']

# Definición de listas de parámetros
pv_multiples = [
   0,
    1,
    2,
    3,
    4,
    
    5,
    
    6,
    
    7,
    
    8,
    10,
    13,
    15
]


bess_multiples = [
 
    1,
    
    2,
    
    3,
    
    4,
    
    5,
    
    6,
    
    7,
    
    8,
    10
]

bess_duration= [
     0,
    1,
    2,
    3,
    4,
    5,
    6,
    7,
     8
]

In [5]:
start_time = time.time()

for curve in optimise_for:
    for pv_multiple in pv_multiples:
        for bess_multiple in bess_multiples:
            for bess_cap in bess_duration:
                try:
                    solar_factor = pv_multiple
                    PessCH = bess_multiple * 3
                    EnESS = PessCH * bess_cap

                    file_path = f'Results/Batch5/CaseB/{curve}_{pv_multiple}_{bess_multiple}_{bess_cap}.csv'
                    profile = pd.read_csv(file_path).fillna(0)
                    profile = profile.astype(int)

                    if curve == 'price':
                        results_econ.loc[len(results_econ)] = get_KPIs(profile, solar_factor=solar_factor, PessCH=PessCH, EnESS=EnESS)
                    elif curve == 'co2':
                        results_env.loc[len(results_env)] = get_KPIs(profile, solar_factor=solar_factor, PessCH=PessCH, EnESS=EnESS)
                except FileNotFoundError:
                    continue

end_time = time.time()
execution_time = end_time - start_time

# Convertir tiempo de ejecución a horas, minutos y segundos
hours, remainder = divmod(execution_time, 3600)
minutes, seconds = divmod(remainder, 60)

to_read = len(pv_multiples) * len(bess_multiples) * len(bess_duration) * len(optimise_for)
been_read = len(results_econ) + len(results_env)

# Imprimir tiempo de ejecución
print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
print(f'Cases to read: {to_read}')
print(f'Cases Read: {been_read}')
print(f'Scenarios Generated: {100 * round((been_read / to_read), 4)}%')

/tmp/ipykernel_492680/485826757.py:105: RuntimeWarning: invalid value encountered in scalar divide
  ress = ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100 if EnESS != 0 else (pv2load / data.Load.sum()) * 100
/tmp/ipykernel_492680/485826757.py:105: RuntimeWarning: invalid value encountered in scalar divide
  ress = ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100 if EnESS != 0 else (pv2load / data.Load.sum()) * 100
/tmp/ipykernel_492680/485826757.py:105: RuntimeWarning: invalid value encountered in scalar divide
  ress = ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100 if EnESS != 0 else (pv2load / data.Load.sum()) * 100
/tmp/ipykernel_492680/485826757.py:105: RuntimeWarning: invalid value encountered in scalar divide
  ress = ((pv2load + bess2load * (pv2bess / (pv2bess + grid2bess))) / data.Load.sum()) * 100 if EnESS != 0 else (pv2load / data.Load.sum()) * 100
/tmp/ipykernel_49268

Execution time: 01:27:23
Cases to read: 1944
Cases Read: 1944
Scenarios Generated: 100.0%


In [6]:
# Funciones adicionales

results_econ['net_savings'] = results_econ['savings'] - results_econ['energy_procurement_costs']
results_env['net_savings'] = results_env['savings'] - results_econ['energy_procurement_costs']

results_econ['net_co2avoided'] = results_econ['co2avoided'] - results_econ['co2burden']
results_env['net_co2avoided'] = results_env['co2avoided'] - results_econ['co2burden']

results_econ['net_earnings'] = results_econ['profit'] + results_econ['savings'] - results_econ['energy_procurement_costs']
results_env['net_earnings'] = results_econ['profit'] + results_env['savings'] - results_econ['energy_procurement_costs']

results_econ['net_abatement'] = results_econ['co2abatement'] + results_econ['co2avoided'] - results_econ['co2burden']
results_env['net_abatement'] = results_econ['co2abatement'] + results_env['co2avoided'] - results_econ['co2burden']

def get_multiples(results):
    results['PV_Load_M'] = results['P_PV'] / 3
    results['BESS_Load_M'] = results['P_BESS'] / 3
    return results

def clean_noStorage(results):
    results.loc[results['T_BESS'] == 0, 'P_BESS'] = 0
    return results

def pv_share(results):
    total_pv = results['pv2load'] + results['pv2bess'] + results['pv2grid'] + results['pv2curtail']
    results['pv2load_share'] = results['pv2load'] / total_pv
    results['pv2bess_share'] = results['pv2bess'] / total_pv
    results['pv2grid_share'] = results['pv2grid'] / total_pv
    results['curtailment_share'] = results['pv2curtail'] / total_pv
    return results

def bess_share(results):
    total_bess = results['bess2grid'] + results['bess2load']
    results['bess2load_share'] = results['bess2load'] / total_bess
    results['bess2grid_share'] = results['bess2grid'] / total_bess
    return results

results_econ = get_multiples(results_econ)
results_env = get_multiples(results_env)

results_econ = clean_noStorage(results_econ)
results_env = clean_noStorage(results_env)

results_econ = pv_share(results_econ)
results_env = pv_share(results_env)

results_econ = bess_share(results_econ)
results_env = bess_share(results_env)

results_econ = results_econ.fillna(value=0)
results_env = results_env.fillna(value=0)

results_econ['PV_Load_M'] = results_econ['PV_Load_M'].round(1)
results_env['PV_Load_M'] = results_env['PV_Load_M'].round(1)
results_econ['BESS_Load_M'] = results_econ['BESS_Load_M'].round(1)
results_env['BESS_Load_M'] = results_env['BESS_Load_M'].round(1)

results_econ[['pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess']] = results_econ[['pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess']].astype('int')
results_env[['pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess']] = results_env[['pv2load', 'pv2bess', 'pv2curtail', 'pv2grid', 'bess2load', 'bess2grid', 'grid2load', 'grid2bess']].astype('int')


'''
P_PV = Paneles fotovoltaicos
P_BESS = Baterías
E_BESS = Energía de las baterías
T_BESS = Duración de las baterías
'''
results_econ = results_econ.drop_duplicates(subset=['P_PV', 'P_BESS', 'E_BESS', 'T_BESS'])
results_env = results_env.drop_duplicates(subset=['P_PV', 'P_BESS', 'E_BESS', 'T_BESS'])


def classify_earnings(value):
    if value > 0:
        return 'positive'
    elif value < 0:
        return 'negative'
    else:
        return 'zero'

def classify_abatement(value):
    if value > 0:
        return 'positive'
    elif value < 0:
        return 'negative'
    else:
        return 'zero'

results_econ['earnings_class'] = results_econ['net_earnings'].apply(classify_earnings)
results_env['earnings_class'] = results_env['net_earnings'].apply(classify_earnings)

results_econ['co2abatement_class'] = results_econ['net_abatement'].apply(classify_abatement)
results_env['co2abatement_class'] = results_env['net_abatement'].apply(classify_abatement)

results_econ['savings_class'] = results_econ['net_savings'].apply(classify_earnings)
results_env['savings_class'] = results_env['net_savings'].apply(classify_earnings)

results_econ['co2avoided_class'] = results_econ['net_co2avoided'].apply(classify_abatement)
results_env['co2avoided_class'] = results_env['net_co2avoided'].apply(classify_abatement)

results_econ['label'] = 'Economic Dispatch Optimisation'
results_env['label'] = 'Environmental Dispatch Optimisation'

results = pd.merge(results_econ, results_env, how='outer')


In [7]:
results_econ.to_csv('Results/Batch5/CaseB/results_econ.csv', sep=';', index=False)
results_env.to_csv('Results/Batch5/CaseB/results_env.csv', sep=';', index=False)
results.to_csv('Results/Batch5/CaseB/results.csv', sep=';', index=False)